# <b> Model Building

In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [4]:
#Read the csv file
df = pd.read_csv('tel_churn.csv')

In [6]:
df.head(5)

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29,29,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56,1889,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53,108,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42,1840,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70,151,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 52 columns):
 #   Column                                   Non-Null Count  Dtype
---  ------                                   --------------  -----
 0   Unnamed: 0                               7032 non-null   int64
 1   SeniorCitizen                            7032 non-null   int64
 2   MonthlyCharges                           7032 non-null   int64
 3   TotalCharges                             7032 non-null   int64
 4   Churn                                    7032 non-null   int64
 5   gender_Female                            7032 non-null   int64
 6   gender_Male                              7032 non-null   int64
 7   Partner_No                               7032 non-null   int64
 8   Partner_Yes                              7032 non-null   int64
 9   Dependents_No                            7032 non-null   int64
 10  Dependents_Yes                           7032 non-null   int64
 11  Phon

In [22]:
#Drop the Unnamed:0 column
df = df.drop('Unnamed: 0', axis=1)

In [26]:
df.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29,29,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56,1889,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53,108,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42,1840,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70,151,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [32]:
#Creating X and Y vaiable
x=df.drop('Churn',axis=1)
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29,29,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56,1889,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53,108,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42,1840,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70,151,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84,1990,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7028,0,103,7362,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7029,0,29,346,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7030,1,74,306,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [36]:
y = df['Churn']
print(y)

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64


## <b> Train Test Split

In [41]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

## <b> Decision Tree Classifier

In [46]:
model_dt = DecisionTreeClassifier(criterion = 'gini',random_state = 100 , max_depth = 6 , min_samples_leaf = 8)

In [48]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [50]:
y_pred = model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [52]:
model_dt.score(x_test,y_pred)

1.0

In [58]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1038
           1       0.62      0.48      0.54       369

    accuracy                           0.79      1407
   macro avg       0.72      0.69      0.70      1407
weighted avg       0.77      0.79      0.78      1407



###### As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

###### Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

###### Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [64]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x, y)

In [68]:
xr_train, xr_test, yr_train, yr_test = train_test_split(X_resampled, y_resampled, test_size = 0.2)

In [74]:
model_dt_smote = DecisionTreeClassifier(criterion = "gini",random_state = 100, max_depth = 6, min_samples_leaf = 8)

In [76]:
model_dt_smote.fit(xr_train,yr_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [80]:
yr_predict = model_dt_smote.predict(xr_test)

In [92]:
model_smote_score = model_dt_smote.score(xr_test,yr_test)
model_smote_score

0.9349522983521249

In [88]:
print(classification_report(yr_test,yr_predict))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       526
           1       0.94      0.94      0.94       627

    accuracy                           0.93      1153
   macro avg       0.93      0.93      0.93      1153
weighted avg       0.93      0.93      0.93      1153



In [90]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[490  36]
 [ 39 588]]


####  <b> Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.

## <b> Random Forest Classifier

In [100]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
model_rf = RandomForestClassifier(
    n_estimators=100,        # Number of trees in the forest
    criterion='gini',        # Use Gini impurity to measure split quality
    random_state=100,        # Ensures consistent results (sets a seed)
    max_depth=6,             # Limits the depth of each tree to 6 levels
    min_samples_leaf=8       # Each leaf must have at least 8 samples (prevents overfitting)
)


In [106]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [108]:
y_pred = model_rf.predict(x_test)

In [110]:
model_rf.score(x_test,y_test)

0.7967306325515281

In [112]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1038
           1       0.66      0.46      0.54       369

    accuracy                           0.80      1407
   macro avg       0.74      0.69      0.71      1407
weighted avg       0.78      0.80      0.78      1407



In [114]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

In [116]:
xr_train1,xr_test1,yr_train1,yr_test1 = train_test_split(X_resampled1,y_resampled1,test_size=0.2)

In [118]:
model_rf_smote = RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [120]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [122]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [126]:
model_rf_smote_score = model_rf_smote.score(xr_test1,yr_test1)
model_rf_smote_score

0.9381443298969072

In [128]:
print(classification_report(yr_test1, yr_predict1))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93       548
           1       0.92      0.97      0.94       616

    accuracy                           0.94      1164
   macro avg       0.94      0.94      0.94      1164
weighted avg       0.94      0.94      0.94      1164



In [130]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[495  53]
 [ 19 597]]


### <b> RF classifier seems to perform better than Decision Tree on this dataset.

#### <b> Pickling the model - Saving the model

In [151]:
import pickle

In [153]:
filename = 'model.sav'

In [155]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [157]:
load_model = pickle.load(open(filename, 'rb'))

In [159]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [161]:
model_score_r1

0.9381443298969072

##### <b> Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.
